This notebook creates, trains, and initializes a chatbot using the bag-of-words model. 


https://chatbotsmagazine.com/contextual-chat-bots-with-tensorflow-4391749d0077

In [2]:
import nltk, pickle, json, re, string, tflearn, spacy, warnings
warnings.filterwarnings("ignore")
from os import path, name, system
from nltk.stem.lancaster import LancasterStemmer
from random import choice, randint, uniform
import numpy as np 
import tensorflow as tf


In [3]:
with open('cs_prompts.json') as file:
    data = json.load(file)

In [4]:
# Preprocess and format the training data
def preprocess_train_data(data):
    stemmer = LancasterStemmer()

    words = []
    labels = list(data.keys())
    docs_x = []
    docs_y = []

    for label in labels:
        for pattern in data[label]['patterns']:
            tokens = nltk.word_tokenize(pattern)
            words.extend(tokens)
            docs_x.append(tokens)
            docs_y.append(label)

    # Pass over punctuation tokens
    ignored_tokens = [',', '.', '?', '!']
    words = [stemmer.stem(w.lower()) for w in words if w not in ignored_tokens]

    words = sorted(set(words))
    labels = sorted(labels)

    training = []
    output = []

    # Template for the BOW
    out_empty = list(np.zeros(len(labels)))

    for x, doc in enumerate(docs_x):
        bag = []
        stemmed = [stemmer.stem(w) for w in doc]

        for w in words:
            if w in stemmed:
                bag.append(1)
            else:
                bag.append(0)

        output_row = out_empty[:]
        output_row[labels.index(docs_y[x])] = 1

        training.append(bag)
        output.append(output_row)  

    training = np.array(training)
    output = np.array(output)    
    
    return words, labels, training, output

In [6]:
if path.exists('./data.pickle'):
    # If a pickle file of the processed training data exists, then it will be loaded
    with open('data.pickle', 'rb') as file:
        words, labels, training, output = pickle.load(file)
else:
    # If no pickle file exists, the training data will be processed and saved in a pickle file
    words, labels, training, output = preprocess_train_data(data)
    with open('data.pickle', 'wb') as file:
        pickle.dump((words, labels, training, output), file)

In [7]:
# Run this cell to create and train a new model
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.fit(training, output, n_epoch=500, batch_size=8, show_metric=True)
model.save('model.tflearn')


Training Step: 3999  | total loss: 0.00635 | time: 0.054s
| Adam | epoch: 500 | loss: 0.00635 - acc: 1.0000 -- iter: 56/60
Training Step: 4000  | total loss: 0.00645 | time: 0.063s
| Adam | epoch: 500 | loss: 0.00645 - acc: 1.0000 -- iter: 60/60
--
INFO:tensorflow:C:\Users\owner\Documents\GitHub\CustomerServiceBot-RW\cs-bagofwords\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [5]:
# Run this cell to load a previously trained model

physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(output[0]), activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net)
model.load('model.tflearn')


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from C:\Users\owner\Documents\GitHub\CustomerServiceBot-RW\cs-bagofwords\model.tflearn


In [8]:
def clear(): 
    # Uses os.system and os.name
    # for windows 
    if name == 'nt': 
        _ = system('cls') 
    # for mac/linux 
    else: 
        _ = system('clear') 

def bag_of_words(s, words, stemmer):
    # Creates a bag of words from a given sequence of tokens
    bag = list(np.zeros(len(words)))
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i]=1
    return np.array(bag)


In [9]:
def chat():
    with open("user_contacts.json") as h_file:
        user_contacts = json.load(h_file)
    CHAT_ENDED = False
    user_name = '' 
    user_phone = ''
    greeting = choice(data['greeting']['responses'])
    stemmer = LancasterStemmer()
    ner = spacy.load('en_core_web_sm')

    def filter_punctuation(s):
        # Uses regular expressions to filter non-alphabetical characters from strings
        regex = re.compile('[%s]' % re.escape(string.punctuation))
        return regex.sub('', s)

    def classify(user_input):
        results = model.predict([bag_of_words(user_input, words, stemmer)])
        result_index=np.argmax(results)
        result_max = np.max(results)
        prediction = labels[result_index]
        responses = data[prediction]['responses']
        response = choice(responses)
        return prediction, response

    def end_chat(inp):
#         print('Continue chat?: ', inp)
        if inp.lower() in ['end', 'quit', 'stop']:
            CHAT_ENDED = True
            return True
        return False

    def get_confirmation(inp):
#         print('Confirmed?: ', inp)
        prediction, response = classify(inp.lower())
        print(f"Bot: {response}")
        if prediction == 'confirm':
            return True, prediction, response
        else:
            return False, prediction, response   

    clear()

    print(f"Bot: {greeting}")

    while not CHAT_ENDED:
        prediction = None
        inp = filter_punctuation(input("You:").lower())

        end_chat(inp)

        if CHAT_ENDED:
            break

        prediction, response = classify(inp)

        print(f"Bot: {response}")

        if prediction in ['open_account', 'close_account']:
            confirmation_inp = filter_punctuation(input("You:").lower())
            confirmed, confirmed_pred, confirmed_resp = get_confirmation(confirmation_inp)

            if end_chat(confirmation_inp):
                break

            if confirmed:
                # User wants to open/close account
                print(f"Bot: {data[prediction]['confirmed']}")
                if user_name == '':
                    print("Bot: Please give me your first and last name")
                    inp = input("You:")

                    if end_chat(inp):
                        break
                    else:
                        parts = ner(inp)
                        helper = []
                        for part in parts:
                            if part.pos_ == 'PROPN':
                                helper.append(part.text)
                        user_name = ' '.join(helper)
                        print(f"Bot: Your name is {user_name}. Is that correct?")
                        confirmation_inp = filter_punctuation(input("You:").lower())

                        if end_chat(confirmation_inp):
                            break
                            
                        confirmed, confirmed_pred, confirmed_resp = get_confirmation(confirmation_inp)

                        if not confirmed:
                            user_name = ''
                            continue
                
                
                if user_name not in user_contacts.keys():
                    user_contacts[user_name] = {'phone':user_phone,'requests':[]}
                else:
                    user_phone = user_contacts[user_name]['phone']
                
                if user_phone == '':
                    print("Bot: Please give me your phone number")
                    inp = input("You:")

                    if end_chat(inp):
                        break
                    else:
                        parts = ner(inp)
                        helper = []
                        for part in parts:
                            if part.pos_ == 'NUM':
                                helper.append(part.text)
                        user_phone = ''.join(helper)
                        print(f"Bot: Your phone number is {user_phone}. Is that correct?")
                        confirmation_inp = filter_punctuation(input("You:").lower())

                        if end_chat(confirmation_inp):
                            break
                            
                        confirmed, confirmed_pred, confirmed_resp = get_confirmation(confirmation_inp)
                        
                        if not confirmed:
                            user_phone = ''
                            continue
                    if user_phone != user_contacts[user_name]['phone']:
                        user_contacts[user_name]['phone'] = user_phone
                        
                user_contacts[user_name]['requests'].append(prediction)
                print(f"Bot: A request has been logged for {user_name} to '{prediction.replace('_', ' ')}'")
                user_name = ''
                user_phone = ''
                prediction = 'continue?'

            else:
                # User does not want to open/close account
                pass

        elif prediction == 'account_balance':
            confirmation_inp = filter_punctuation(input("You:").lower())
            if end_chat(confirmation_inp):
                break
            confirmed, confirmed_pred, confirmed_resp = get_confirmation(confirmation_inp)
            if confirmed:
                balance = round(uniform(1,9999999), 2)
                print(f"Bot: Your account balance is ${balance}")
                prediction = 'continue?'
            else:
                pass
            
        elif prediction == 'goodbye':
            break

        elif prediction == '':
            print("Bot: I'm sorry, I didn't quite get that.")
        
        if prediction == 'continue?':
            print("Bot: Is there anything else I can help with?")

    with open("user_contacts.json", "w") as h_file:
        json.dump(user_contacts, h_file)

In [10]:
chat()

Bot: Hello. Welcome to BotBank. How can I help?


You: I want to open a new account.


Bot: You are trying to open an account. Is that correct?


You: Yes


Bot: Great. Thanks for confirming.
Bot: You will need to speak with a representative, but I can log a request to speed up the process.
Bot: Please give me your first and last name


...

Bot: Thank you.
Bot: A request has been logged for Lester Morrison to 'close account'
Bot: Is there anything else I can help with?


You: What are your hours?


Bot: All BotBank locations are open 7am-4pm Monday-Friday!


You: That's all I needed.


Bot: Glad I could help. Thanks for choosing BotBank Have a nice day.
